In [2]:
import numpy as np
import pandas as pd
import glob
import matplotlib as mpl
from ast import literal_eval
pd.options.display.max_rows = 300
np.set_printoptions(precision = 4, suppress = True)



* <b>Hi</b> 

--- 
separator(---)
* Little dot <b>bold</b> `little box` 

So, we want to have a program that can take multiple csv files, each csv file corresponds to one game. Important information: number of simulations in all decisions, winner, c-parameter for selection and c-parameter for final choice.

This should be the final column of all csv files.

The program should be able to extract this row and make a new table out of it.


It should also collect all MC moves and answers. Each move is a column. Move 1 move 2.... we're not interested in the answers of trades?


Anyways: first: visualize one csv table.

In [3]:
def replace(rw):
    if rw.iloc[0] == '1' or rw.iloc[0] == '2' or rw.iloc[0] == '3' or rw.iloc[0] == '4':
        rw.iloc[0] = literal_eval(rw.iloc[1])[1][0]
        rw.iloc[1] = literal_eval(rw.iloc[1])[0]
        rw.iloc[2] = 'Monte Carlo'
        #print(rw) 
    elif rw.iloc[0] == 'Na':
        rw.iloc[3] = literal_eval(rw.iloc[1])[0]
        rw.iloc[0] = literal_eval(rw.iloc[1])[1]
        rw.iloc[1] = np.NaN
        rw.iloc[2] = 'Monte Carlo A'
        #print(rw)
    elif literal_eval(rw.iloc[0])[0] == 'Random move answer 2x2':
        rw.iloc[3] = literal_eval(rw.iloc[1])[0]
        rw.iloc[1] = np.NaN
        rw.iloc[0] = 'Trade 2x2'
        rw.iloc[2] = 'Random'    
        #print(rw)
    elif literal_eval(rw.iloc[0])[0][0] == 'Random move answer 3x1':
        rw.iloc[3] = literal_eval(rw.iloc[0])[1][0]
        rw.iloc[0] = 'Trade 3x1'
        rw.iloc[2] = 'Random'
        #print(rw)
    elif literal_eval(rw.iloc[0])[0] == 'Random move 3x1 trade':
        rw.iloc[0] = 'Trade 3x1'
        rw.iloc[1] = literal_eval(rw.iloc[1])
        rw.iloc[2] = 'Random'
        #print(rw)
    elif literal_eval(rw.iloc[0])[0] == 'Random move 2x2 trade':
        rw.iloc[0] = 'Trade 2x2'
        rw.iloc[1] = literal_eval(rw.iloc[1])
        rw.iloc[2] = 'Random'
        #print(rw)
    elif literal_eval(rw.iloc[0])[0][0] == 'Random move Discard': 
        rw.iloc[1] = literal_eval(rw.iloc[0])[1]
        rw.iloc[0] = 'Discard'
        rw.iloc[2] = 'Random'
        #print(rw)
    elif literal_eval(rw.iloc[0])[0][0] == 'Random move Store':   
        rw.iloc[1] = literal_eval(rw.iloc[0])[1]
        rw.iloc[0] = 'Store'
        rw.iloc[2] = 'Random'
        #print(rw)
    else:
        print('??')
        print(literal_eval(rw.iloc[0])[0])
        print(type(literal_eval(rw.iloc[0])))

    return rw

In [4]:
def actually(tbl):
    new = pd.DataFrame({'Move': [],  'Cards': [], 'Player': [], 'Choice': []})
    hoi = tbl.iloc[0]
    how_are_you = tbl.iloc[1:13, 0:4].rename(columns={'a': 'Move', 'b': 'Cards', 'c': 'Player', 'd': 'Choice'})
    for index, row in how_are_you.iterrows():
        b = replace(row)
        new = pd.concat([new, b.to_frame().T])
    bye = tbl.iloc[13]
    return new, bye

In [5]:
csv_filess = glob.glob('actual/*.csv')
df_list = []
mc_list = []
wins = pd.DataFrame({})

for csv_file in csv_filess:
    df = pd.read_csv(csv_file)
    print(csv_file)
    moves, winner = actually(df)
    wins = pd.concat([wins, winner.to_frame().T])
    df_list.append(moves)

for frame in df_list:
    mc = frame.groupby(by="Player")
    mc_list.append(mc.get_group('Monte Carlo'))


actual\actual_moves20240614_164219.csv
actual\actual_moves20240614_165201.csv
actual\actual_moves20240614_170036.csv
actual\actual_moves20240614_170358.csv
actual\actual_moves20240614_193049.csv
actual\actual_moves20240615_143003.csv
actual\actual_moves20240615_145530.csv
actual\actual_moves20240615_154255.csv
actual\actual_moves20240615_163220.csv
actual\actual_moves20240615_165750.csv
actual\actual_moves20240615_172141.csv
actual\actual_moves20240615_174610.csv
actual\actual_moves20240615_181158.csv
actual\actual_moves20240615_183555.csv
actual\actual_moves20240615_185940.csv
actual\actual_moves20240615_192352.csv
actual\actual_moves20240615_194759.csv
actual\actual_moves20240615_201223.csv
actual\actual_moves20240615_210246.csv
actual\actual_moves20240615_212740.csv
actual\actual_moves20240615_215203.csv
actual\actual_moves20240615_221612.csv
actual\actual_moves20240615_230714.csv
actual\actual_moves20240615_235658.csv
actual\actual_moves20240616_002232.csv
actual\actual_moves202406

In [6]:
wins_count = wins.groupby('b').count()

print(wins_count['a'])

b
Monte Carlo Expand    137
Random 2               24
Name: a, dtype: int64


In [7]:
mc_list

[        Move                 Cards       Player Choice
 1  Trade 2x2  [[2r, 4g], [4g, 5b]]  Monte Carlo    NaN
 4    Discard              [5b, 5b]  Monte Carlo    NaN
 7      Store                  [4g]  Monte Carlo    NaN
 9  Trade 3x1          [4g, 3b, 2p]  Monte Carlo    NaN,
          Move                 Cards       Player Choice
 1   Trade 2x2  [[4g, 5b], [3o, 2y]]  Monte Carlo    NaN
 4   Trade 3x1          [2r, 3b, 5b]  Monte Carlo    NaN
 8     Discard              [3b, 2y]  Monte Carlo    NaN
 11      Store                  [5b]  Monte Carlo    NaN,
          Move                 Cards       Player Choice
 1   Trade 2x2  [[4g, 2y], [2p, 3o]]  Monte Carlo    NaN
 5       Store                  [3b]  Monte Carlo    NaN
 8   Trade 3x1          [4g, 3o, 2r]  Monte Carlo    NaN
 11    Discard              [5b, 4g]  Monte Carlo    NaN,
         Move                 Cards       Player Choice
 1  Trade 2x2  [[5b, 2p], [2y, 4g]]  Monte Carlo    NaN
 5      Store                  [2r]

In [8]:

order = pd.DataFrame({})
chaos = [[], []]

for mc in mc_list:
    order = pd.concat([order, mc['Move'].to_frame().reset_index(drop=True).T])

    if mc['Move'][1] == 'Trade 2x2':
        frst = sorted(mc['Cards'].iloc[0][0])
        secnd = sorted(mc['Cards'].iloc[0][1])
        if frst[1] > secnd[1]:   
            chaos[0].append(sorted(mc['Cards'].iloc[0][1]))
            chaos[1].append(sorted(mc['Cards'].iloc[0][0]))

        else:
            chaos[0].append(sorted(mc['Cards'].iloc[0][0]))
            chaos[1].append(sorted(mc['Cards'].iloc[0][1]))


In [9]:
order.head()

,0,1,2,3
Move,Trade 2x2,Discard,Store,Trade 3x1
Move,Trade 2x2,Trade 3x1,Discard,Store
Move,Trade 2x2,Store,Trade 3x1,Discard
Move,Trade 2x2,Store,Discard,Trade 3x1
Move,Trade 2x2,Store,Discard,Trade 3x1


In [10]:
def move_count(move_order):
    first_move = move_order.groupby(0).count()[1].to_frame()
    second_move = move_order.groupby(1).count()[2].to_frame()
    third_move = move_order.groupby(2).count()[3].to_frame()
    fourth_move = move_order.groupby(3).count()[0].to_frame()
    counted_df = pd.concat([first_move, second_move, third_move, fourth_move], axis=1).fillna(0)
    counted_df = counted_df.rename(columns={1: 'First Move', 2: 'Second Move', 3: 'Third Move', 0: 'Fourth Move'})
    return counted_df  #print nicely?
    counted_df.style \
    .format(precision=3, thousands=".", decimal=",") \
    .format_index(str.upper, axis=1) 


moves_counted = move_count(order)
moves_counted
print(moves_counted.to_latex(float_format="{:.0f}".format))




\begin{tabular}{lrrrr}
\toprule
 & First Move & Second Move & Third Move & Fourth Move \\
\midrule
Discard & 1 & 44 & 52 & 67 \\
Trade 2x2 & 157 & 6 & 1 & 0 \\
Trade 3x1 & 6 & 90 & 31 & 37 \\
Store & 0 & 24 & 80 & 60 \\
\bottomrule
\end{tabular}



In [12]:
def patterns(pars_df):
    df_patterns = [[], [], [], [], []]
    for index, rw in pars_df.iterrows():
        first = rw.iloc[0][0]
        second = rw.iloc[0][1]
        third = rw.iloc[1][0]
        fourth = rw.iloc[1][1]
        tracker = [0, 0, 0, 0, 0]
        if first == second:  #low pair
            tracker[0] = 1
            if second == third or second == fourth:   #triple
                tracker[2] = 1
        elif third == fourth:  #high pair
            tracker[1] = 1
            if second == third or first == third:   #triple
                tracker[2] = 1
        elif second <= third:  # two low, vs two high  23 34
            tracker[3] = 1
        elif second >= third:  #highlow highlow 
            tracker[4] = 1

        for i in range(0,len(tracker)):
            df_patterns[i].append(tracker[i])

    df_patterns = pd.DataFrame({'Low pair': df_patterns[0], 'High pair': df_patterns[1], 'Triple': df_patterns[2], 'two low, vs two high': df_patterns[3],'highlow highlow': df_patterns[4]})

    return df_patterns

In [13]:

chaotic = pd.DataFrame({'First Offer': chaos[0], 'Second Offer': chaos[1]} )
nw = patterns(chaotic)
pat_counted = nw.sum().to_frame().rename(columns={0: 'Actual Moves'})
print(pat_counted)
new = pd.concat([chaotic, nw], axis = 1)


                      Actual Moves
Low pair                        11
High pair                       15
Triple                           3
two low, vs two high            52
highlow highlow                 79


In [14]:
new.style \
  .format(precision=3, thousands=".", decimal=",") \
  .format_index(str.upper, axis=1) 

,FIRST OFFER,SECOND OFFER,LOW PAIR,HIGH PAIR,TRIPLE,"TWO LOW, VS TWO HIGH",HIGHLOW HIGHLOW
0,"['2r', '4g']","['4g', '5b']",0,0,0,1,0
1,"['2y', '3o']","['4g', '5b']",0,0,0,1,0
2,"['2p', '3o']","['2y', '4g']",0,0,0,0,1
3,"['2y', '4g']","['2p', '5b']",0,0,0,0,1
4,"['2p', '3o']","['5b', '5b']",0,1,0,0,0
5,"['2r', '2y']","['3b', '3o']",0,0,0,1,0
6,"['3b', '3o']","['3o', '5b']",0,0,0,1,0
7,"['2p', '2y']","['4g', '5b']",0,0,0,1,0
8,"['2p', '2y']","['3o', '3o']",0,1,0,0,0
9,"['2y', '3b']","['3b', '4g']",0,0,0,1,0


In [15]:
def get_max(grp):
    lst = []
    for rnd in range(1,7):
        a = grp.get_group(rnd).sort_values(by= 'c-value', ascending=False)
        aa = a.groupby(by='c-value')
        ab = aa.get_group(max(list(aa.groups.keys())))
        lst.append(ab)
    return lst

In [16]:
def todo(tbl):
    grouped = tbl.groupby(by="round")

    gg = get_max(grouped)

    pieces = {'Decision 1': gg[0], 'Decision 2': gg[1], 'Decision 3': gg[2],'Decision 4': gg[3], 'Decision 5': gg[4], 'Decision 6': gg[5]}
    result = pd.concat(pieces).drop(columns='round')
    first = gg[0]
    return result, first

In [17]:

frac_list = []
cards_list = [[], []]
# Get a list of all CSV files in a directory
csv_files = glob.glob('sims/*.csv')

for csv_file in csv_files:
    df = pd.read_csv(csv_file)
    print(csv_file)
    a, b = todo(df)
    for i in b['move']:
        if literal_eval(i)[1] == ['Trade 2x2', 0, 4]:
            frst = sorted(literal_eval(i)[0][0])
            secnd = sorted(literal_eval(i)[0][1])
            if frst[1] > secnd[1]:   
                cards_list[0].append(secnd)
                cards_list[1].append(frst)

            else:
                cards_list[0].append(frst)
                cards_list[1].append(secnd)
    frac_list.append(a)

ord_cards = pd.DataFrame({'First Offer': cards_list[0], 'Second Offer': cards_list[1]} )
nw2 = patterns(ord_cards)
pat2_counted = nw2.sum().to_frame().rename(columns={0: 'Simulation'})
print(pat2_counted)

new2 = pd.concat([ord_cards, nw], axis = 1)


sims\sim_childs20240614_193049.csv
sims\sim_childs20240615_143003.csv
sims\sim_childs20240615_145530.csv
sims\sim_childs20240615_154255.csv
sims\sim_childs20240615_163220.csv
sims\sim_childs20240615_165750.csv
sims\sim_childs20240615_172141.csv
sims\sim_childs20240615_174610.csv
sims\sim_childs20240615_181158.csv
sims\sim_childs20240615_183555.csv
sims\sim_childs20240615_185940.csv
sims\sim_childs20240615_192352.csv
sims\sim_childs20240615_194759.csv
sims\sim_childs20240615_201223.csv
sims\sim_childs20240615_210246.csv
sims\sim_childs20240615_212740.csv
sims\sim_childs20240615_215203.csv
sims\sim_childs20240615_221612.csv
sims\sim_childs20240615_230714.csv
sims\sim_childs20240615_235658.csv
sims\sim_childs20240616_002232.csv
sims\sim_childs20240616_004745.csv
sims\sim_childs20240616_011339.csv
sims\sim_childs20240616_014054.csv
sims\sim_childs20240616_020625.csv
sims\sim_childs20240616_023209.csv
sims\sim_childs20240616_032544.csv
sims\sim_childs20240616_035120.csv
sims\sim_childs20240

In [18]:
new3 = pd.concat([pat_counted, pat2_counted], axis = 1)
new3

,Actual Moves,Simulation
Low pair,11,97
High pair,15,82
Triple,3,17
"two low, vs two high",52,231
highlow highlow,79,422


In [19]:

print(new3.to_latex(float_format="{:.0f}".format))

\begin{tabular}{lrr}
\toprule
 & Actual Moves & Simulation \\
\midrule
Low pair & 11 & 97 \\
High pair & 15 & 82 \\
Triple & 3 & 17 \\
two low, vs two high & 52 & 231 \\
highlow highlow & 79 & 422 \\
\bottomrule
\end{tabular}



In [ ]:
new.style \
  .format(precision=3, thousands=".", decimal=",") \
  .format_index(str.upper, axis=1) 


[]

So we want to analyze if there are fractions with 100% success. Order 
Which moves are these? Are some of these moves more recurrent?

Do 2 simulations per SEED.

Need to check if code is actually functional.

First lets just merge two csv files